# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem. The wanted output is discrete [yes, no] [0,1]  (and binary in this case).
The main goal is to be able to make predictions to know whether a student is going to pass or not.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no'].shape[0]

# Check operations
n_total = n_passed+n_failed

# TODO: Calculate graduation rate
grad_rate = round(100*n_passed)/n_total

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of total: {}".format(n_total)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)
student_data.head()

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Number of total: 395
Graduation rate of the class: 67.09%


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [32]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Set the number of testing points
num_test = X_all.shape[0] - num_train
RANDOM_STATE_VALUE = 42

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,
                                                    test_size=num_test, random_state=RANDOM_STATE_VALUE)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


In [101]:
#import matplotlib.pyplot as plt
#%matplotlib inline

#aux_arr = y_train
#aux_arr[aux_arr == 'yes'] = 1.0
#aux_arr[aux_arr == 'no'] = 0.0
#print y_train

#for feature in X_train:
#    plt.scatter(X_train['absences'][aux_arr == 1.0], X_train[feature][aux_arr == 1.0], color = "b", label="passed")
#    plt.scatter(X_train['absences'][aux_arr == 0.0], X_train[feature][aux_arr == 0.0], color = "r", label="failed")
#    plt.legend()
#    plt.xlabel("absences")
#    plt.ylabel(feature)
#    plt.show()

## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
1. Naive Bayes: Naive Bayes algorithm is simple, fast, efficient and works great with small training sets (high-bias expected). However it is usually outperformed by other algorithms. NB assumes independent features but in this case they are not because for example sex_M and sex_F are clearly dependent.
Even if this assumption is not true, NB still performs well in practice.
Because of our training set isn't too large (300 samples) it seems reasonable to try a simple and efficient algorithm first (Occam's razor).
NB is widely used in spam detection problems.

++: Simple, fast, works well with small datasets, good as first approximation.

--: It is easily outperformed by other algorithms


2. Random Forest: Decision Tree Algorithm is also a good choice because our dataset isn't very large so that huge trees can be avoided. Instead of using a Decision Tree algorithm I prefer to try a Random Forest model in this case. 
Theoretically, Random Forest classifiers have the advantages of Decision Trees (easy to interpret and explain, non-parametric, robust to outliers) but also are more scalable. Besides RF are harder to overfit and easier to tune.
Random forest Model is usually used for categorization of text documents.

++: Non-parametric, doesn't care about data (outliers, linearly separable), scalable, (**review: RF is not a priori faster than Decision Trees. Anyway DT model is not chosen here so it's not possible to compare both algorithms in this particular case.**)

--: Complexity, it is necessary to be careful with dimensionality


3. Gradient Boosting Classifier: Boosting methods iterate N times over the training set to make better predictions based on these N estimations. In theory "boosting is better" because the model can learn "more" from the data. Higher resources consumption is expected in terms of computation. Recently, gradient boosting has gained some popularity in the field of learning to rank. Commercial web search engines such as Yahoo and Yandex use variants of gradient boosting in their machine-learned ranking engines.

++: Good performance, used in real apps.

--: Complexity, training this model is more expensive 

(**review: GBC is chosen for similar reasons than RF was. The dataset given is small and it doesn't have a large number of features so a good performance it's expected choosing this model. In addition, some of the features are just "dummy features" with 2 possible values so in these situations ensemble methods work great.**)


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [29]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [143]:
# TODO: Import the three supervised learning models from sklearn
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import BaggingClassifier
#from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = RandomForestClassifier()
clf_C = GradientBoostingClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
print "CLASSIFIER A"
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_A, X_train, y_train, X_test, y_test)

print "\nCLASSIFIER B"
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_B, X_train, y_train, X_test, y_test)

print "\nCLASSIFIER C"
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_C, X_train, y_train, X_test, y_test)

CLASSIFIER A
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0015 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8467.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.8029.


Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0007 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8406.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7244.


Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0007 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8038.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7634.

CLASSIFIER B
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0259 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.7015.


Training a RandomForestClassifier u

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |                   0.0004|                  0.0003|            0.8467|           0.8029|
| 200               |                   0.0003|                  0.0002|            0.8406|           0.7244|
| 300               |                   0.0003|                  0.0003|            0.8038|           0.7634|

** Classifer 2 - Random Forest Classifier**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |                   0.0009|                  0.0009|            1.0000|           0.7015|
| 200               |                   0.0012|                  0.0008|            1.0000|           0.7500|
| 300               |                   0.0015|                  0.0019|            0.9927|           0.7669|

** Classifer 3 - Gradient Boosting Classifier**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |                   0.0007|                  0.0004|            1.0000|           0.7519|
| 200               |                   0.0009|                  0.0005|            0.9964|           0.7591|
| 300               |                   0.0013|                  0.0005|            0.9739|           0.7794|

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **:
Choosing a SVM model should be avoided because it is probably more complex and less efficient in terms of memory usage (time to make a prediction).

Although Naive Bayes could be a right choice (simple, fast, efficient) we can't tune any parameter to improve the performance so it is discarded.

Under real conditions I would probably choose Naive Bayes if we **didn't expect that the dataset is going to increase**. With the current 300-samples set it's an optimal choice.

However, as we want to go deeper and try to improve the actual performance I will choose Gradient Boosting Classifier. It seems that the default parameters are causing overfitting so maybe we can make better predictions tuning them.

Training a Boosting Algorithm is more expensive (x100 t_training) than a Naive Bayes one so as I said, in general, it could be better to choose Naive Bayes for this type of problem.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **: Boosting methods iterate N times over the dataset to train weak learners. The final output of the boosting method is a weighted sum of all of the weak learners outputs. These weak models are usually standard Decision Trees.

For example, if we have 4 estimations of a particular datapoint, 3 label_a and 1 label_b, the final output could be label_a because 3 of the 4 learners have predicted label_a. In general these algorithms are more complex and the individual outputs are weighted. Besides that, a particular weak estimator (i) focuses more on datapoints that weren't properly classified by the previous estimator (i-1)

It is said that "boosting is better" because you can learn "more" from the original training dataset and because of that your model will generalize better.

Based on the actual data we are now able to make correct predictions of 4 out of 5 students just knowing some features about them. Our model is a big ensemble of some trained decision trees that make assumptions based on mathematical conditions deduced from our data.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [144]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
# Random Forest Params
#parameters = [{'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]}]
# Gradient Boosting Classifier Params
parameters = [{'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20], 
              'min_samples_split': [5, 10, 15, 20, 25, 50],
              'learning_rate': [1.0]}]
# KNN Params
#parameters = [{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30]}]
# SVC Params
#parameters = [
#    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
#    {'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]},
#    {'kernel': ['poly'], 'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}]

# TODO: Initialize the classifier
#clf = RandomForestClassifier(min_samples_split=20)
clf = GradientBoostingClassifier()
#clf = KNeighborsClassifier()
#clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print clf.get_params()

Made predictions in 0.0003 seconds.
Tuned model has a training F1 score of 0.8618.
Made predictions in 0.0005 seconds.
Tuned model has a testing F1 score of 0.7971.
{'presort': 'auto', 'loss': 'deviance', 'verbose': 0, 'subsample': 1.0, 'max_leaf_nodes': None, 'learning_rate': 1.0, 'warm_start': False, 'min_samples_leaf': 1, 'n_estimators': 3, 'min_samples_split': 20, 'init': None, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'max_features': None, 'max_depth': 3}


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **:
F1 Score (train): 0.8618
F1 Score (test): 0.7971
t_prediction (train): 0.0003s
t_prediction (test): 0.0003s

F1 score goes from 0.7794 to 0.7971 (+2.5% approximately) and the prediction time is actually similar so the final performance is improved "for free" because we are not using more resources, just tuning the parameters of the model.

In any case, as I said for a "fast", simple and cheap solution the algorithm that seems to fit better is Naive Bayes.
Having a small dataset, NB is very close to the "Best Model" in terms of performance and efficiency.


SOURCES:

https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms

https://www.youtube.com/watch?v=UHBmv7qCey4

Udacity Forum and videos :)

**Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.